In [1]:
from cnn_globals import *

from cnn_data import cnn_data, cnn_data_split

%matplotlib inline

[INFO    ] Using cmssw CMSSW_10_1_7
[INFO    ] Using numpy 1.14.1
/cvmfs/cms.cern.ch/slc6_amd64_gcc630/external/py2-pippkgs_depscipy/3.0-omkpbe5/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
[INFO    ] Using tensorflow 1.5.0
Using TensorFlow backend.
[INFO    ] Using keras 2.1.4
[INFO    ] .. list devices: [_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456)]
[INFO    ] Using scipy 1.0.0
[INFO    ] Using sklearn 0.19.2


In [2]:
# Import images and labels
#data = cnn_data_split(infile_images, test_size=0.4, shuffle=False, nentries=80000)
data = cnn_data_split(infile_images, test_size=0.4, shuffle=False, nentries=3742000)

(images_px_train, images_px_test, images_ch_train, images_ch_test, labels_train, labels_test, parameters_train, parameters_test) = data

[INFO    ] Loading cnn data from ../test7/histos_tbe.17.npz ...
[INFO    ] Loaded the images with shape (3742978, 50, 2),(3742978, 50, 2)
[INFO    ] Loaded the labels with shape (3742978, 3)
[INFO    ] Loaded the parameters with shape (3742978, 4)
[INFO    ] Loaded # of training and testing events: (2245200, 1496800)


In [3]:
import cnn_data_gen
reload(cnn_data_gen)

<module 'cnn_data_gen' from 'cnn_data_gen.pyc'>

In [4]:
from cnn_data_gen import ImageDataGenerator
from six.moves import zip

def parse_image_fn(pixels, channels):
  n = pixels.shape[0]
  assert(pixels.shape == (n,2))
  assert(channels.shape == (n,n_channels))

  bad_pixel = -99
  mask = (pixels[:,0] != bad_pixel)

  image_shape = (n_rows, n_columns, n_channels)
  image = np.zeros(image_shape, dtype=channels.dtype)
  #image[pixels[mask,0], pixels[mask,1]] = channels[mask]
  image[pixels[mask,0], pixels[mask,1]] = np.column_stack((channels[mask,0],1-channels[mask,0]))
  return image

def parse_label_fn(labels):
  assert(labels.shape == (3,))
  lb = labels[:1]
  lb = keras.utils.to_categorical(lb[0], num_classes=n_classes)  # in Keras, use one-hot encoding
  return lb

fake_input_shape = [images_px_train.shape[0], 1, 1, 1]
images_ph_train = np.zeros(fake_input_shape, dtype=np.float32)
x_train = (images_ph_train, images_px_train, images_ch_train)
y_train = labels_train

datagen = ImageDataGenerator(featurewise_center=False,
                             samplewise_center=False,
                             featurewise_std_normalization=False,
                             samplewise_std_normalization=False,
                             validation_split=0.1)

datagen.preprocessing_x_fn = parse_image_fn
datagen.preprocessing_y_fn = parse_label_fn

datagen_seed = 2023
datagen_batch_size = 100
#datagen.fit(x_train, seed=datagen_seed)
train_generator = datagen.flow(x_train, y_train, subset='training', batch_size=datagen_batch_size, shuffle=True, seed=datagen_seed)
val_generator = datagen.flow(x_train, y_train, subset='validation', batch_size=datagen_batch_size, shuffle=True, seed=datagen_seed)

#model.fit_generator(train_generator, steps_per_epoch=None, epochs=5, workers=1, validation_data=val_generator, validation_steps=None)